In [1]:
import sys, os
import pandas as pd
import numpy as np

In [2]:
trainingDataFilename = 'trainingSet.csv'
testDataFilename = 'testSet.csv'
modelIdx = 2

In [3]:
def sigmoid(scores):
    predictions = np.zeros(len(scores))
    for i in range(len(predictions)):
        if scores[i] >= 0:
            predictions[i] +=  1.0 / (1.0 + np.exp(-scores[i]))
        else:
            predictions[i] += np.exp(scores[i]) / (1.0 + np.exp(scores[i]))
    return predictions

def lr(trainingSet, testSet):
    print len(trainingSet.columns)
    regularization = 0.01
    step_size = 0.01
    
    max_iterations = 500
    tol = 1e-6
    
    count = 0
    diff = 1.0
    
    train_labels = trainingSet['decision']
    test_labels = testSet['decision']
    
    trainingSet = trainingSet.drop('decision', axis=1)
    testSet = testSet.drop('decision', axis=1)
    
    #print train_labels, trainingSet
    w = np.zeros(len(trainingSet.columns) + 1)
    
    # Add intercept
    X = np.array(trainingSet)
    Y = np.array(train_labels)
    intercept = np.ones((X.shape[0], 1))
    #X = np.concatenate((X, intercept.T), axis=1)
    X = np.hstack((X, intercept))
    
    while(count < max_iterations and diff > tol):
        count += 1
        norm_old = np.linalg.norm(w)
        
        scores = np.dot(X, w)
        predictions = sigmoid(scores)

        gradient = np.dot(X.T, (predictions - Y))

        for j in range(len(w)):
            gradient[j] += regularization * w[j]
            
        #gradient /= len(train_labels)
        w -= step_size * gradient
            
        norm_new = np.linalg.norm(w)
        
        diff = abs(norm_new - norm_old)
        #print w
        print count, diff
    
    return w

In [4]:
def svm(trainingSet, testSet):
    print len(trainingSet.columns)
    regularization = 0.01
    step_size = 0.5
    
    max_iterations = 500
    tol = 1e-6
    
    count = 0
    diff = 1.0
    
    train_labels = trainingSet['decision']
    test_labels = testSet['decision']
    
    trainingSet = trainingSet.drop('decision', axis=1)
    testSet = testSet.drop('decision', axis=1)
    
    w = np.zeros(len(trainingSet.columns) + 1)
    
    # Add intercept
    X = np.array(trainingSet)
    Y = np.array(train_labels)
    
    for i in range(len(Y)):
        if Y[i] == 0:
            Y[i] = -1.0
        else:
            Y[i] = 1.0
    
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((X, intercept))
    
    while(count < max_iterations and diff > tol):
        count += 1
        norm_old = np.linalg.norm(w)
        
        predictions = np.dot(X, w)

        gradient = np.zeros(X.shape[1])
        for i in range(len(predictions)):
            if predictions[i] * Y[i] < 1.0:
                gradient -= Y[i] * X[i]
            
        gradient /= len(train_labels)
        #print gradient.shape
        
        for j in range(len(w) - 1):
            gradient[j] += 1.0 * regularization * w[j]

        #gradient /= len(train_labels)
        w -= step_size * gradient
        norm_new = np.linalg.norm(w)
        
        diff = abs(norm_new - norm_old)
        print count, diff
    
    return w

In [5]:
trainingSet = pd.read_csv(trainingDataFilename)
testSet = pd.read_csv(testDataFilename)

if modelIdx == 1:
    w = lr(trainingSet, testSet)
else:
    w = svm(trainingSet, testSet)

261
1 3.3390134775333355
2 5.102754375694117
3 1.7424639012953653
4 1.082706474824259
5 4.40950742290708
6 6.251203971240096
7 9.54483690793526
8 8.99377302443336
9 6.952230114911771
10 4.536711086625102
11 2.206644962245921
12 0.7626290062937056
13 2.256147557604649
14 5.3751560845740745
15 5.546985262636933
16 6.768573716125725
17 6.213986063382415
18 1.474639274023926
19 3.277340751504049
20 3.7568138378250318
21 6.719150773238768
22 6.0475515157692445
23 4.0800176320092625
24 2.1444537006715425
25 0.8416070412852203
26 1.6013577850836427
27 2.0779656339494075
28 4.7761526756133605
29 4.363502332903405
30 0.2056918274983488
31 3.2713164940234734
32 1.495677706804031
33 0.5396242570613481
34 1.7099495347727647
35 1.9301067780512025
36 4.366583501566483
37 3.8729286268549306
38 0.2731474026424614
39 0.6699386629508677
40 0.7916263378592898
41 1.28968268725985
42 1.502788870869642
43 3.8180439820245358
44 3.3778275976779497
45 0.04005166743330335
46 2.2626640655676056
47 0.637774011800

364 0.3292735146542043
365 0.37292730819359576
366 0.3269929364455919
367 0.37381439702070196
368 0.32833047326536047
369 0.36867480805107533
370 0.32482486499519325
371 0.37078553477081755
372 0.3261830277371729
373 0.36810605103075744
374 0.3251257694460108
375 0.36663593587982035
376 0.3240796286374348
377 0.3651826928071742
378 0.32423918749358904
379 0.3636736337672204
380 0.3243209809267924
381 0.3610348903944498
382 0.323330676589066
383 0.35968365044885786
384 0.32346278882915414
385 0.3594571432119693
386 0.32359167815771883
387 0.358012010715953
388 0.32250097197457706
389 0.356585322752899
390 0.32142309134873415
391 0.3551746664250004
392 0.3203578687365223
393 0.3561410433687229
394 0.32166188509864213
395 0.3535027432294129
396 0.31936920096702437
397 0.35327061443307883
398 0.3206285989927622
399 0.35182608539701477
400 0.31950913830890215
401 0.35040237781954175
402 0.31969938733154635
403 0.3502886724584684
404 0.3198864568035873
405 0.3501761050251275
406 0.3200717688

In [8]:
def get_accuracy_lr(w, trainingSet, testSet):
    total_train = len(trainingSet)
    count_train = 0
    total_test = len(testSet)
    count_test = 0
    
    train_labels = trainingSet['decision']
    test_labels = testSet['decision']
    
    trainingSet = trainingSet.drop('decision', axis=1)
    testSet = testSet.drop('decision', axis=1)
    
    # Training accuracy
    X = np.array(trainingSet)
    Y = np.array(train_labels)
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((X, intercept))
    
    scores = np.dot(X, w)
    predictions = sigmoid(scores)
        
    for i in range(len(predictions)):
        if predictions[i] > 0.5:
            predictions[i] = 1
        else:
            predictions[i] = 0

    for i in range(len(predictions)):    
        if predictions[i] == int(train_labels[i]):
            count_train += 1

    training_accuracy = 1.0 * count_train/total_train
    print 'Training Accuracy LR:', '%.2f' % training_accuracy
    
    # Test accuracy
    X = np.array(testSet)
    Y = np.array(test_labels)
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((X, intercept))

    scores = np.dot(X, w)
    predictions = sigmoid(scores)

    for i in range(len(predictions)):
        if predictions[i] > 0.5:
            predictions[i] = 1
        else:
            predictions[i] = 0

    for i in range(len(predictions)):    
        if predictions[i] == int(test_labels[i]):
            count_test += 1
            
    test_accuracy = 1.0 * count_test/total_test
    print 'Test Accuracy LR:', '%.2f' % test_accuracy
    
def get_accuracy_svm(w, trainingSet, testSet):
    total_train = len(trainingSet)
    count_train = 0
    total_test = len(testSet)
    count_test = 0
    
    train_labels = trainingSet['decision']
    test_labels = testSet['decision']
    
    trainingSet = trainingSet.drop('decision', axis=1)
    testSet = testSet.drop('decision', axis=1)
    
    # Training accuracy
    X = np.array(trainingSet)
    Y = np.array(train_labels)
    print Y
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((X, intercept))
    
    predictions = np.dot(X, w)
        
    for i in range(len(predictions)):
        if predictions[i] >= 0.0:
            predictions[i] = 1
        else:
            predictions[i] = 0

    for i in range(len(predictions)):    
        if predictions[i] == int(train_labels[i]):
            count_train += 1

    training_accuracy = 1.0 * count_train/total_train
    print 'Training Accuracy SVM:', '%.2f' % training_accuracy
    
    # Test accuracy
    X = np.array(testSet)
    Y = np.array(test_labels)
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((X, intercept))

    predictions = np.dot(X, w)

    for i in range(len(predictions)):
        if predictions[i] >= 0.0:
            predictions[i] = 1
        else:
            predictions[i] = 0

    for i in range(len(predictions)):    
        if predictions[i] == int(test_labels[i]):
            count_test += 1
            
    test_accuracy = 1.0 * count_test/total_test
    print 'Test Accuracy SVM:', '%.2f' % test_accuracy

In [9]:
if modelIdx == 1:
    get_accuracy_lr(w, trainingSet, testSet)
else:
    get_accuracy_svm(w, trainingSet, testSet)

[1 1 1 ... 1 0 0]
Training Accuracy SVM: 0.56
Test Accuracy SVM: 0.55
